## Setup environment

In [ ]:
!pip install openai

Setup your chatgpt key

In [ ]:
import os

# Prompt the user for their API key
api_key = input("Please enter your OpenAI API key: ")

# Store the API key in the environment variable
os.environ["OPENAI_API_KEY"] = api_key

## Setup simulation

In [ ]:
class GridSimulation:
    def __init__(self):
        # Initialize the grid as a 4x4 list of lists filled with 0s
        self.grid = [[0 for _ in range(4)] for _ in range(4)]
        # Place the user agent at row 1, column 1 (0-indexed)
        self.user_row = 0
        self.user_col = 0
        # Set the goal at row 4, column 4 (3-indexed)
        self.goal_row = 3
        self.goal_col = 3
        self.simulation_completed = False
        self.update_grid()

    def update_grid(self):
        """Updates the grid to reflect the current positions of the user agent and goal."""
        if self.user_row == self.goal_row and self.user_col == self.goal_col:
            self.simulation_completed = True
        else:
            for r in range(4):
                for c in range(4):
                    self.grid[r][c] = 0  # Reset grid
            self.grid[self.user_row][self.user_col] = 1  # Mark the user agent's position
            self.grid[self.goal_row][self.goal_col] = 2  # Mark the goal's position

    def move(self, direction):
        """Moves the user agent in the specified direction if the move is legal."""

        if self.simulation_completed:
            return "Simulation already completed."

        new_row, new_col = self.user_row, self.user_col

        if direction == "left":
            new_col -= 1
        elif direction == "right":
            new_col += 1
        elif direction == "top":
            new_row -= 1
        elif direction == "bottom":
            new_row += 1
        else:
            return "Invalid direction. Please choose left, right, top, or bottom."

        # Check if the new position is within the grid boundaries
        if 0 <= new_row < 4 and 0 <= new_col < 4:
            # Update the user's position
            self.user_row, self.user_col = new_row, new_col
        else:
            # Illegal move, do not update the user's position
            return "Illegal move: Cannot move outside the grid."

        # Place the user agent in the new position
        self.grid[self.user_row][self.user_col] = 1
        self.grid[self.goal_row][self.goal_col] = 2  # Ensure the goal's position remains marked with a 2

        self.update_grid()
        return self.get_state()

    def get_state(self):
        """Returns the current state of the grid."""
        if self.simulation_completed:
            return "Simulation completed."
        else:
            return '\n'.join([' '.join([str(cell) for cell in row]) for row in self.grid])

## Setup agent to run the simulation

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_state",
            "description": "Retrieves the current state of the grid simulation. If the simulation is not completed, it returns a visual representation of the grid with the user agent and goal positions. If the simulation is completed, it returns a message indicating completion.",
            "parameters": {}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "move",
            "description": "Moves the user agent in a specified direction on the grid. The method updates the user agent's position and checks for a win condition. If the move is illegal or outside the grid boundaries, it returns an error message. If the move leads to the user agent reaching the goal, it completes the simulation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "direction": {
                        "type": "string",
                        "enum": ["left", "right", "top", "bottom"],
                        "description": "The direction in which to move the user agent. Valid options are 'left', 'right', 'top', or 'bottom'."
                    }
                },
                "required": ["direction"]
            }
        }
    }
]

def run_conversation(messages, simulation):

    runs = []
    ideas = []
    complete = False

    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # print(response_message)
    ideas.append(response_message)

    if tool_calls:
        available_functions = {
            "get_state": simulation.get_state,
            "move": simulation.move
        }
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            # print(function_args, function_args.get("direction"), function_to_call)

            if function_name == "move":
              function_response = function_to_call(
                  direction=function_args.get("direction"),
              )
            else:
              function_response = function_to_call()

            runs.append(function_response)

            if function_response == "Simulation completed." or function_response == "Simulation already completed.":
              complete = True

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

    return messages, runs, complete, ideas

## Run simulation until complete

In [ ]:
messages = [{"role": "user", "content": "Complete the simulation by taking 1 to 2 with the given tools."}]

In [ ]:
simulation = GridSimulation()

In [ ]:
for i in range(0,7):
  print("## Thinking for step", i, "\n")
  messages, runs, complete, ideas = run_conversation(messages, simulation)
  for idea in ideas:
    print("## ChatGPT thought\n")
    print(idea.content)
  for run in runs:
    print("## Simulation step\n")
    print(run)
  if complete:
    break